In [ ]:
# Carga de librerias
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import glob
import os
from PIL import Image
from itertools import chain
from collections import Counter
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.preprocessing.image import DirectoryIterator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score
from keras import backend as K
from itertools import chain
# Definición de las variables globales
IMG_SIZE=32
NUM_CHANNELS=3
NUM_CLASSES=43

In [ ]:
# Declaración de los directorios
current_dir=os.getcwd()
dataset_dir=current_dir+"/Dataset"
dir_train=dataset_dir+"/Train"
train_data=pd.read_csv(dataset_dir+"/Train.csv")
test_data=pd.read_csv(dataset_dir+"/Test.csv")
meta_data=pd.read_csv(dataset_dir+"/Meta.csv")

x_images=test_data["Path"]
x_train_images=train_data["Path"]
x_train_labels=train_data[['ClassId']].to_numpy()
y_test=test_data[['ClassId']].values


In [ ]:
def change_brightness(img, value):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v,value)
    v[v > 255] = 255
    v[v < 0] = 0
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
def test_data(x_images, IMG_SIZE,brig):
    data=[]
    for i in x_images:
        try:
            image=cv2.imread(dataset_dir+"/"+i)
            image=change_brightness(image,brig)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((IMG_SIZE, IMG_SIZE))
            data.append(np.array(size_image))
        except AttributeError:
            print("Error")
    X_test=np.array(data)
    X_test=X_test.astype('float32')/255
    return X_test

In [ ]:
settings_trained_models = {
    'cnn_92': {
        'conv': [64,64,64],
        'dens': [256,256],
        'name': "cnn_92"
    },
    'cnn_74': {
        'conv': [16,64,64],
        'dens': [128,128],
        'name': "cnn_74"
    },
    'cnn_17': {
        'conv': [32,64,64],
        'dens': [],
        'name': "cnn_17"
    },
    'cnn_82': {
        'conv': [32,32,64],
        'dens': [256,256],
        'name': "cnn_82"
    },

    'cnn_54': {
        'conv': [16,16,32],
        'dens': [128,128],
        'name': "cnn_54"
    },
    'cnn_67': {
        'conv': [16,32,64],
        'dens': [256,256],
        'name': "cnn_67"
    },
    'cnn_91': {
        'conv': [64,64,64],
        'dens': [128],
        'name': "cnn_91"
    },
    'cnn_61': {
        'conv': [16,32,32],
        'dens': [128],
        'name': "cnn_61"
    },
    'cnn_66': {
        'conv': [16,32,64],
        'dens': [128],
        'name': "cnn_66"
    },
    'cnn_84': {
        'conv': [32,32,64],
        'dens': [128,128],
        'name': "cnn_84"
    },
       
}

In [ ]:
def predictions(settings, X_test):
    results=[]
    for name, model_settings in settings.items():
        h5_model=current_dir+'/Results/CNN_Train/'+name+'.h5'
        model=tf.keras.models.load_model(h5_model)
        pred=np.argmax(model.predict(X_test), axis=-1)
        val_acc=accuracy_score(y_test,pred)
        result={"Model": name,
          "Accuracy": val_acc
            }
        results.append(result)
    results_table=pd.DataFrame(results)
    return results_table


In [ ]:
X_test_180=test_data(x_images, IMG_SIZE,180)
results_180=predictions(settings_trained_models, X_test_180)
results_180.to_csv(current_dir+'/Results/CNN_Attack/CNN_10_results_attack_180.csv')

X_test_80=test_data(x_images, IMG_SIZE,-80)
results_80=predictions(settings_trained_models, X_test_80)
results_80.to_csv(current_dir+'/Results/CNN_Attack/CNN_10_results_attack_80.csv')

X_test_1=test_data(x_images[0:int(len(x_images)/2)], IMG_SIZE,180)
X_test_2=test_data(x_images[int(len(x_images)/2):], IMG_SIZE,-80)
X_test_combined=np.concatenate((X_test_1,X_test_2))

results_combined=predictions(settings_trained_models, X_test_combined)
results_combined.to_csv(current_dir+'/Results/CNN_Attack/CNN_10_results_attack_combined.csv')
